In [1]:
# 전처리
import pickle
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

def load_cifar_batch(filename):
    with open(filename, 'rb') as file:
        batch = pickle.load(file, encoding='latin1')
    # 이미지 데이터 재배열: [num_samples, 3, 32, 32]
    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']
    return features, labels

def create_datasets(data_paths, test_path):
    # 데이터를 로드하고 하나의 큰 훈련 데이터셋으로 결합
    train_features = []
    train_labels = []
    for path in data_paths:
        features, labels = load_cifar_batch(path)
        train_features.append(features)
        train_labels.append(labels)
    train_features = np.concatenate(train_features)
    train_labels = np.concatenate(train_labels)
    
    # 테스트 데이터 로드
    test_features, test_labels = load_cifar_batch(test_path)

    # numpy 배열을 PyTorch 텐서로 변환
    train_features = torch.tensor(train_features).permute(0, 3, 1, 2).float() / 255.0
    train_labels = torch.tensor(train_labels).long()
    test_features = torch.tensor(test_features).permute(0, 3, 1, 2).float() / 255.0
    test_labels = torch.tensor(test_labels).long()
    
    # TensorDataset 생성
    train_dataset = TensorDataset(train_features, train_labels)
    test_dataset = TensorDataset(test_features, test_labels)
    return train_dataset, test_dataset

# 파일 경로 설정
data_paths = [f'cifar-10-batches-py/data_batch_{i}' for i in range(1, 6)]
test_path = 'cifar-10-batches-py/test_batch'

# 데이터셋 생성
train_dataset, test_dataset = create_datasets(data_paths, test_path)

# DataLoader 생성 예
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
# 학습
import torch
from torch import nn, optim

In [ ]:
from model import ModelClass

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ModelClass()
model.to(device)

criteration = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

epochs = 10
# 학습 로직
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for data, targets in train_loader:
        data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criteration(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}, Loss: {total_loss / len(train_loader)}")

    # 성능 테스트
    model.eval()
    total_correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == targets).sum().item()
            total += targets.size(0)
    accuracy = 100 * total_correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")